In [1]:
from keras.models import Model, Sequential
from keras.layers import Dense
import numpy as np
import random
from tqdm import tqdm
from typing import List, Dict

from src.model.players.GeneticBot import GeneticBot
from src.GameController import GameController
from src.model.Game import Game
from src.view.NoGraphics import NoGraphics

In [2]:
blank_model = GeneticBot("").model

def model_to_genotype(model):
    weights = model.get_weights()
    # flatten weights
    res = []
    for layer in weights:
        res.extend(layer.flatten())
    return res

def genotype_to_model(genotype:list):
    model = blank_model
    weights = []
    i = 0
    for layer in model.get_weights():
        layer_size = layer.size
        new_layer = np.array([genotype[i:i+layer_size]]).reshape(layer.shape)
        weights.append(new_layer)
        i += layer_size
    model.set_weights(weights)
    return model

GENOTYPE_SIZE = len(model_to_genotype(blank_model))
print(GENOTYPE_SIZE)

def random_genotype():
    return [random.uniform(-1,1) for _ in range(GENOTYPE_SIZE)]

10049


In [3]:
class Individual:
    def __init__(self, genotype):
        self.genotype = genotype
        self.player = GeneticBot(str(genotype), delay=0.0001)
        self.player.set_model(genotype_to_model(genotype))
        self.fitness = 0

def history_to_fitness(history:List[Dict[str, Dict[str, int]]], population:List[Individual]) -> None:
    for individual in population:
        individual.fitness = history[-1][individual.player.name]["score"]

def crossover(parent1:Individual, parent2:Individual) -> Individual:
    genotype1 = parent1.genotype
    genotype2 = parent2.genotype
    split_point = random.randint(0, GENOTYPE_SIZE)
    child_genotype = genotype1[:split_point] + genotype2[split_point:]
    return Individual(child_genotype)

def mutate(individual:Individual) -> None:
    mutation_rate = 0.1
    for i in range(GENOTYPE_SIZE):
        if random.random() < mutation_rate:
            individual.genotype[i] = random.uniform(-1,1)

In [4]:

def genetic_algorithm(output_file:str):
    population = [Individual(random_genotype()) for _ in range(50)]
    best_individual = None
    for generation in range(10):
        with open(output_file, "a") as f:
            f.write(f"Generation {generation}\n")
        # make batches of 5 players
        batches = [population[i:i+5] for i in range(0, len(population), 5)]
        for batch in tqdm(batches):
            game = Game([individual.player for individual in batch], False)
            game_controller = GameController(game, NoGraphics())
            game_controller.run()
            history = game_controller.get_history()
            history_to_fitness(history, batch)
        
        population.sort(key=lambda x: x.fitness, reverse=True)
        with open(output_file, "a") as f:
            f.write(f"  Best genotype: {population[0].genotype}\n")
            f.write(f"  Best fitness: {population[0].fitness}\n")
        best_individual = population[0]
        new_population = []
        for _ in range(len(population)//2):
            parent1 = random.choice(population)
            parent2 = random.choice(population)
            child = crossover(parent1, parent2)
            mutate(child)
            new_population.append(child)
        population = new_population
    return best_individual


In [5]:
best_individual = genetic_algorithm()

print(f"Best individual: {best_individual.fitness}")

Generation 0


  0%|          | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 15ms/step


 10%|█         | 1/10 [00:26<03:55, 26.22s/it]

1/1 [==============================] - 0s 20ms/step
ccc

...

100%|██████████| 10/10 [04:24<00:00, 26.46s/it]

  Best fitness: 310


Generation 1


  0%|          | 0/5 [00:00<?, ?it/s]

1/1 [==============================] - 0s 14ms/step


In [ ]:
# save model
model = best_individual.player.model
model.save("best_model.h5")